In [ ]:

# Generalized Code for Grad-CAM (Gradient-weighted Class Activation Mapping) Visualisation

# 1. Importing Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# 2. Loading Dataset Function
# For Grad-CAM, we would typically work with image datasets. 
# Hence, this function is a placeholder and might need adjustments based on the actual dataset format.
def load_dataset(file_path):
    # Placeholder function
    pass

# 3. Preprocessing Function
# Adjust this function based on the image dataset and the required input format for the model.
def preprocess_data(images, labels):
    # Placeholder function
    pass

# 4. Model Training
def train_model(X_train, y_train):
    # For demonstration, we'll use a simple CNN model
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')  # Assuming 5 classes for this example
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
    return model

# 5. Grad-CAM Implementation
# This function is a placeholder. Actual implementation would require handling model layers and gradients.
def apply_gradcam(model, instance):
    # TODO: Implement Grad-CAM using appropriate methods.
    pass

# 6. Main Execution
if __name__ == "__main__":
    # Adjust loading and preprocessing based on the actual dataset
    images, labels = load_dataset('path_to_your_image_dataset')
    X_train, X_test, y_train, y_test = preprocess_data(images, labels)
    model = train_model(X_train, y_train)
    
    # Applying Grad-CAM on a sample instance
    sample_instance = X_test[0]
    apply_gradcam(model, sample_instance)

# 5. Grad-CAM Implementation
def generate_gradcam_heatmap(model, img_array, last_conv_layer_name, classifier_layer_names):
    # First, we create a model that maps the input image to the activations of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)
    
    # Second, we create a model that maps the activations of the last conv layer to the final class predictions
    classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = tf.keras.Model(classifier_input, x)
    
    # Compute the gradient of the class channel with regard to the feature map
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel in the feature map array by "how important this channel is"
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    
    return heatmap

# 6. Displaying Grad-CAM
def display_gradcam(img, heatmap, alpha=0.4):
    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = plt.cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    return superimposed_img

# 7. Main Execution
if __name__ == "__main__":
    # Placeholder for loading data and preprocessing
    X_train, y_train = None, None
    model = train_model(X_train, y_train)
    
    # Placeholder for selecting a sample image for Grad-CAM visualization
    img_array = None

    # Generate Grad-CAM heatmap
    heatmap = generate_gradcam_heatmap(model, img_array, 'conv2d', ['flatten', 'dense', 'dense_1'])
    superimposed_img = display_gradcam(img_array[0], heatmap)

    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()
